In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import os
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn import metrics

pd.set_option('display.max_colwidth',100000) #https://stackoverflow.com/questions/54692405/output-truncation-in-google-colab
pd.set_option('display.max_rows', 100000)

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

In [ ]:
basic_table = pd.read_pickle('Downloads/basic_table.pkl')
apsiii = pd.read_csv('Downloads/apsiii_score.csv')
oasis = pd.read_csv('Downloads/oasis_scores.csv')
sofa = pd.read_csv('Downloads/sofa_score.csv')
sirs = pd.read_csv('Downloads/sirs_score.csv')

In [ ]:
oasis.columns.values

In [ ]:
aspiii = apsiii.drop(['Unnamed: 0', 'hr_score', 'mbp_score', 'temp_score',
       'resp_rate_score', 'pao2_aado2_score', 'hematocrit_score',
       'wbc_score', 'creatinine_score', 'uo_score', 'bun_score',
       'sodium_score', 'albumin_score', 'bilirubin_score',
       'glucose_score', 'acidbase_score', 'gcs_score'], axis=1, inplace = True)
oasis =oasis.drop(['Unnamed: 0'], axis=1)

In [ ]:
apsiii.columns.values

In [ ]:
scores = apsiii.merge(oasis, how='inner', on='stay_id')
illness_severity = basic_table.merge(scores, how='inner', on='stay_id')
illness_severity.head(5)

In [ ]:
print(len(illness_severity))
illness_severity.columns.values

In [ ]:
illness_severity = illness_severity.drop(['los', 'subject_id_x_y', 'hadm_id_x', 'subject_id_y', 'hadm_id_y'], axis=1,)

In [ ]:
print(type(illness_severity))

In [ ]:
illness_severity = illness_severity.sort_values(by=['label'])

In [ ]:
illness_severity.head(5
                     )

In [ ]:
# illness_severity.to_excel('Downloads/illness_severity.xlsx')

In [ ]:
sirs = sirs.drop(['Unnamed: 0'], axis=1)
sofa = sofa.drop(['Unnamed: 0'], axis=1)
sirs.columns.values

In [ ]:
sir = basic_table.merge(sirs, how='left', on='stay_id')
print(len(sir))

In [ ]:
sofa.head(5)

In [ ]:
from datetime import datetime
def convert_time(x):
  date_time_str = x
  if date_time_str[-1] == ':':
    date_time_str += '00'
  date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
  return date_time_obj
sofa.starttime = sofa.starttime.apply(convert_time)
basic_table.pO2_time = basic_table.pO2_time.apply(convert_time)

In [ ]:
time = pd.DataFrame({'stay_id': basic_table['stay_id'], 'pO2_time': basic_table['pO2_time'],
                     'dicom':basic_table['dicom'] })
phys = sofa.merge(time, how='right', on='stay_id')
phys['time_diff'] = phys.pO2_time - phys.starttime
phys['time_diff'] = phys['time_diff'].abs()
phys['time_diff'] = phys['time_diff'] / pd.Timedelta(minutes=1)
phys['min_time_diff'] = phys.groupby('dicom')['time_diff'].transform('min')

phys = phys.where(phys['time_diff'] == phys['min_time_diff'])

phys = phys[phys['dicom'].notna()]


In [ ]:
phys = phys.drop_duplicates(subset= 'dicom',ignore_index=True)
phys = phys.reset_index()
phys = phys.drop(['index', 'Unnamed: 0'], axis=1)

In [ ]:
print(len(phys))
print(phys['dicom'].nunique())
print(phys['stay_id'].nunique())

In [ ]:
sofa_sirs = sir.merge(phys, how='inner', on='dicom')

In [ ]:
sofa_sirs = sofa_sirs.drop(['index', 'time_diff', 'min_time_diff'], axis=1)

In [ ]:
print(len(sofa_sirs))
print(sofa_sirs['dicom'].nunique())
print(sofa_sirs['stay_id_x'].nunique())

In [ ]:
sofa_sirs.head(50)

In [ ]:
# sofa_sirs.to_excel('Downloads/sofa_sirs.xlsx')

In [ ]:
print(type(basic_table['tsne_features'].iloc[0]))

In [ ]:
tsne_features= basic_table[['tsne_features_xs','tsne_features_ys']].to_numpy()
print(type(tsne_features[:,0]))

In [ ]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1,11):
    kmeans = KMeans(n_clusters=i, init = "k-means++", 
                   max_iter= 300, n_init=10)
    kmeans.fit(tsne_features)
    wcss.append(kmeans.inertia_)
plt.plot(range(1,11), wcss, color='#377eb8')
# plt.title("Elbow Method")
plt.xlabel("Number of clusters", weight='bold', labelpad=10, size= 'x-large')
plt.ylabel("Within-Cluster Sum of Square\n(x10^6)", weight='bold', labelpad=10, size='x-large')
plt.axvline(x=4, color='#e41a1c', linestyle='--', linewidth='1')
plt.axvline(x=6, color='#e41a1c', linestyle='--', linewidth='1')
plt.xticks([0,1,2,3,4,5,6,7,8,9,10])
plt.gca().get_xticklabels()[4].set_color("#e41a1c")
plt.gca().get_xticklabels()[4].set_weight("bold")
plt.gca().get_xticklabels()[5].set_color("#e41a1c")
plt.gca().get_xticklabels()[5].set_weight("bold")
plt.gca().get_xticklabels()[6].set_color("#e41a1c")
plt.gca().get_xticklabels()[6].set_weight("bold")
# plt.ticklabel_format(style='plain')
# plt.savefig('Downloads/Elbow_method_tsne2.png', format='png', dpi=1200,bbox_inches='tight')
plt.show()

In [ ]:
# Select the 0th feature: xs
xs = tsne_features[:,0]

# Select the 1st feature: ys
ys = tsne_features[:,1]

In [ ]:
from sklearn.cluster import KMeans
model = KMeans(n_clusters=4)
model.fit(tsne_features)
label = model.predict(tsne_features)

In [ ]:
plt.scatter(xs,ys, alpha = 0.5, s=10, c=label,marker ='.')
# plt.savefig('Downloads/c4_kmeans_tsne.png', format='png', dpi=1200)
plt.show()

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

X= tsne_features
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=10)
    cluster_labels = clusterer.fit_predict(X)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
        X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )
#     plt.savefig('Downloads/kmeans_on_raw_data_silhouette_score_c='+ str(n_clusters)+'.png')
plt.show()
# for i in plt.get_fignums():
#     plt.figure(i).savefig('/wherever/files/shall/roam/figure' + str(i) + '.png')